In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt

/Users/amlannag/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

In [3]:
X = lfw_people.images
Y = lfw_people.target
print("X_min:",X.min(),"X_train_max:", X.max())
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
X_train = X_train[:, :, :,np.newaxis]
X_test = X_test[:, :, :,np.newaxis]
print("X_train shape:", X_train.shape)

X_min: 0.0 X_train_max: 1.0
X_train shape: (901, 50, 37, 1)


In [4]:
n_samples, h, w = lfw_people.images.shape
n_classes = len(np.unique(y_train))
batch_size = 256
epochs = 1
categories = n_classes

In [5]:
inputs = Input(shape=(h,w,1))
net1 = Conv2D(filters = 32, kernel_size = (3,3), padding="same", activation="relu")(inputs)
pool1 = MaxPooling2D(pool_size=(2,2))(net1)
net2 = Conv2D(filters = 32, kernel_size = (3,3), padding="same", activation="relu")(pool1)
pool2 = MaxPooling2D(pool_size=(2,2))(net2)
flat = Flatten()(pool2)
net3 = Dense(128, activation="relu")(flat)
output = Dense(categories, activation="softmax")(net3)

In [6]:
y_train_cat = to_categorical(y_train, categories)
y_test_cat = to_categorical(y_test, categories)

In [11]:
model = Model(inputs, output)

model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train_cat,
                epochs=5,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(X_test, y_test_cat))

Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.6610 - loss: 0.9635 - val_accuracy: 0.5349 - val_loss: 1.1833
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.5831 - loss: 1.0113 - val_accuracy: 0.7158 - val_loss: 0.9325
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 142ms/step - accuracy: 0.7764 - loss: 0.8199 - val_accuracy: 0.6667 - val_loss: 0.9788
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 209ms/step - accuracy: 0.7126 - loss: 0.8141 - val_accuracy: 0.7339 - val_loss: 0.8474
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 193ms/step - accuracy: 0.7633 - loss: 0.7412 - val_accuracy: 0.7339 - val_loss: 0.8366


In [12]:
predictions = model.predict(X_test)
predictions = np.argmax(predictions, axis=1)
print(predictions)

#print performance to compare with previous RF versions
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
[3 3 6 3 3 3 4 1 3 3 3 4 3 4 3 3 3 3 3 4 1 6 3 3 1 1 3 3 3 3 3 3 3 3 3 3 3
 3 3 1 3 1 3 1 1 1 4 3 4 3 3 3 3 3 6 2 1 3 3 3 1 1 1 4 3 5 6 4 6 3 6 6 3 3
 3 2 1 6 3 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 1 3 1 1 6 6 3 3 3 1 3 3 3 3 3 1 3
 1 6 4 3 3 3 3 1 3 1 3 3 0 3 4 4 3 1 3 6 6 6 4 4 6 3 3 1 6 2 3 3 1 3 4 1 3
 6 1 1 6 1 3 3 3 6 3 1 0 6 3 3 1 3 1 5 1 3 3 1 1 3 1 4 3 3 2 2 3 3 3 3 6 3
 3 3 3 3 2 3 2 3 2 6 3 3 6 3 6 3 2 1 2 3 2 6 2 1 2 1 3 3 3 3 4 3 3 0 3 3 3
 1 1 6 3 3 3 6 3 3 3 1 3 3 1 6 6 2 3 3 3 2 2 3 0 3 3 3 4 4 3 3 4 3 4 3 1 3
 3 3 3 6 1 3 4 1 1 4 6 1 1 3 3 3 3 3 3 3 3 1 3 3 3 1 0 3 3 3 4 3 3 3 3 3 1
 4 5 3 6 3 5 6 2 1 3 6 3 3 3 3 3 1 6 3 3 3 1 3 2 3 2 2 1 2 3 4 6 3 6 6 4 2
 3 6 1 6 4 3 3 3 3 6 3 1 6 6 3 4 1 6 3 4 5 3 1 3 1 6 3 3 2 3 3 1 0 3 6 6 3
 3 3 3 6 4 3 1 0 1 3 6 3 3 3 6 6 2]
              precision    recall  f1-score   support

           0       0.71      0.28      0.40        18
           1       0.86      0.80      0.83        69
           2

In [13]:
model2 = Model(inputs, output)


model2.compile(
    optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.0005,
    momentum=0.9,
    nesterov=False,
    name="SGD"), 
    loss='categorical_crossentropy', 
               metrics=['accuracy'])

model2.fit(X_train, y_train_cat,
                epochs=20,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(X_test, y_test_cat))



Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 267ms/step - accuracy: 0.7943 - loss: 0.6849 - val_accuracy: 0.7287 - val_loss: 0.8319
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 189ms/step - accuracy: 0.7861 - loss: 0.6922 - val_accuracy: 0.7313 - val_loss: 0.8206
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 191ms/step - accuracy: 0.7950 - loss: 0.6796 - val_accuracy: 0.7442 - val_loss: 0.8087
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 195ms/step - accuracy: 0.7984 - loss: 0.6572 - val_accuracy: 0.7442 - val_loss: 0.8043
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 197ms/step - accuracy: 0.8029 - loss: 0.6539 - val_accuracy: 0.7545 - val_loss: 0.7983
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 196ms/step - accuracy: 0.8140 - loss: 0.6318 - val_accuracy: 0.7571 - val_loss: 0.7922
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 194ms/step - accuracy: 0.8119 - loss: 0.6267 - val_accuracy: 0.7519 - val_loss: 0.7889
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 199ms/step - accuracy: 0.8081 - loss: 0.6313 - val_accuracy: 0.7494 - val_loss:

In [14]:
predictions = model2.predict(X_test)
predictions = np.argmax(predictions, axis=1)
print(predictions)

#print performance to compare with previous RF versions
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
[3 3 6 3 3 3 4 1 3 3 3 4 3 4 3 3 3 3 3 4 1 0 3 3 1 1 0 3 3 3 3 3 3 3 3 3 3
 3 3 1 3 1 3 1 1 1 4 3 4 3 3 3 3 3 6 2 1 3 3 3 1 1 1 4 0 5 6 4 1 3 6 6 3 3
 3 2 1 6 3 3 3 1 4 3 3 3 3 3 3 3 3 6 3 3 1 1 1 1 6 6 3 3 3 1 3 3 3 3 3 1 3
 1 6 4 3 3 3 3 1 3 1 3 3 0 3 4 4 3 1 3 6 6 6 5 4 6 3 3 1 6 2 3 3 1 3 6 1 3
 6 1 1 1 1 3 3 3 6 3 1 0 6 0 3 1 3 1 5 1 3 3 1 1 3 1 5 3 3 2 2 3 3 3 3 6 3
 3 3 3 3 2 3 2 3 2 6 3 3 6 3 6 3 2 1 2 3 2 6 2 1 2 1 3 3 3 3 4 3 3 0 1 3 3
 1 1 6 3 3 3 1 3 3 3 1 0 3 1 6 6 2 3 3 5 2 2 3 0 3 3 3 4 4 3 3 4 3 2 3 1 3
 3 3 3 6 1 3 4 1 1 3 6 1 1 3 3 3 3 3 3 3 3 1 1 3 3 1 0 3 3 3 3 3 3 3 1 3 1
 4 5 3 6 3 5 6 2 1 3 1 3 1 3 3 3 1 6 3 3 3 1 3 2 3 2 2 1 2 5 4 6 3 6 6 5 2
 3 6 1 6 4 3 3 2 3 6 3 1 6 1 3 4 1 6 3 4 5 3 1 3 1 6 3 3 2 3 0 1 0 3 6 1 3
 3 3 3 6 4 3 1 0 1 3 1 3 3 3 6 6 2]
              precision    recall  f1-score   support

           0       0.54      0.39      0.45        18
           1       0.82      0.91      0.86        69
           2